# DM-1766 Demo: Before

*This notebook should be run with the HSC stack (circa hscPipe 3.8.1).*

We'll start by reading a file written with an older version of the pipeline (actually, a modern version of the HSC pipeline, which has older afw.table and measurement libraries).  This is an empty catalog - it has no records, just  a Schema - but it's the full schema for single-frame processing in the HSC stack, and as such it pretty much represents the state of the LSST pipeline back before we started the measurement framework overhaul.  We're using this exact file in the new unit tests in afw that were written for DM-1766.

In [1]:
import os
import lsst.afw.table

catalog = lsst.afw.table.SourceCatalog.readFits("empty-v0.fits")
schema = catalog.schema

Here's the full schema:

In [2]:
print schema

Schema(
    (Field['L'](name="id", doc="unique ID"), Key<L>(offset=0, nElements=1)),
    (Field['Coord'](name="coord", doc="position in ra/dec", units="IRCS; radians"), Key<Coord>(offset=8, nElements=2)),
    (Field['L'](name="parent", doc="unique ID of parent source"), Key<L>(offset=24, nElements=1)),
    (Field['Flag'](name="flags.negative", doc="set if source was detected as significantly negative"), Key['Flag'](offset=32, bit=0)),
    (Field['I'](name="deblend.nchild", doc="Number of children this object has (defaults to 0)"), Key<I>(offset=40, nElements=1)),
    (Field['Flag'](name="deblend.deblended-as-psf", doc="Deblender thought this source looked like a PSF"), Key['Flag'](offset=32, bit=1)),
    (Field['PointD'](name="deblend.psf-center", doc="If deblended-as-psf, the PSF centroid"), Key<PointD>(offset=48, nElements=2)),
    (Field['D'](name="deblend.psf-flux", doc="If deblended-as-psf, the PSF flux"), Key<D>(offset=64, nElements=1)),
    (Field['Flag'](name="deblend.too-many-

There are a few things worth pointing out here:
 - The field names here mostly use `.` as both a word separator and a namespace separator, with all the characters lowercase.  There are a few that use `-` as a word separator.  In the new measurement framework, we've switched to using `_` as a namespace character, with `camelCase` to separate words.  The main advantage here is that the field names are now both valid Python tokens and valid SQL field names.
 - While most of the fields are scalars (`"L"==long`, `"I"=int`, `"D"=double`) or `Flag`s, we also have a few compound fields: `Coord`, `PointD`, `CovPointF`, `MomentsD`, `CovMomentsF`.  Those let you store complex objects in records.  On the LSST side, those field types have been deprecated for a while, and we'd stopped using them in the new measurement framework (I'll demo what's replaced them in a bit).  But we were still using some of them in some key places even in the LSST side - notably `Coord` fields were being used in the "minimal schema" that all `SourceRecord` schemas start with.
 - This doesn't show us the slots, because those are saved with the table object, not the schema, in older versions of the pipeline.  We can see the definitions of them individually like this:

In [3]:
print catalog.table.getPsfFluxDefinition(), catalog.table.getCentroidDefinition()

flux.psf centroid.sdss
